In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
# In this notebook, we try to fit a model of growth constant.
# The dataset consists of bended rectangles, done by applying an implicit module of order 1 with known linear growth law.
# To generate the dataset, simply execute "python3 generate_bending.py bendings.pickle nb_of_elem_in_dataset".
# We then fit a polynomial model of growth constants of order 2.

In [3]:
# Load our data
data = []
with open("bendings2.pickle", 'rb') as f:
    data = pickle.load(f)

template = data[0][0][:-1]
template = torch.unique_consecutive(template, dim=0)


true_implicit1_points = data[0][1]
true_C = data[0][2]
bendings = data[1:]

# Make sure each points in each data element are consecutively unique (needed when dealing varifold attachment atm).
dataset = [torch.unique_consecutive(target, dim=0)[:-1]  for target in list(zip(*bendings))[0]]
#dataset = [target + 0.1*torch.rand(target.shape) for target in dataset]
targets = dataset[:15]

print("Dataset size: {size}".format(size=len(targets)))

Dataset size: 10


In [4]:
%matplotlib qt5

# Display our data set and template
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
for target in targets:
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), color='grey', lw=2)
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
plt.plot(true_implicit1_points[:, 0].numpy(), true_implicit1_points[:, 1].numpy(), 'x')
plt.axis('equal')
plt.show()

In [7]:
# Defines the implicit module of order 1 that will be used by the atlas.
sigma_implicit1 = 1.5

implicit1_points = true_implicit1_points
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)
C_init = torch.ones(implicit1_points.shape[0], 2, 1)
implicit1 = dm.DeformationModules.ImplicitModule1(2, true_implicit1_points.shape[0], sigma_implicit1, C_init, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()))


In [ ]:
# Learning model: polynomial of order 2
# The polynomial model we will try to fit on our deformation constants
def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

# Callback called by the atlas. Serves as glue for our model of deformation constants.
def callback_compute_c_2(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    modules[1]._ImplicitModule1Base__C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

# Initial coefficient for the polynomial model of the deformation constants
abc_init = torch.zeros(6, 2)
abc_init[0] = torch.ones(2)    

In [8]:
# Learning model: polynomial of order 3 
# The polynomial model we will try to fit on our deformation constants
def pol_order_3(pos, a, b, c, d, e, f, g, h, i, j):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1] + g*pos[:, 0]**3 + h*(pos[:, 0]**2)*pos[:, 1] + i*pos[:, 0]*(pos[:, 1]**2) + j*pos[:, 1]**3

# Callback called by the atlas. Serves as glue for our model of deformation constants.
def callback_compute_c_3(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    g = abc[6].unsqueeze(1)
    h = abc[7].unsqueeze(1)
    i = abc[8].unsqueeze(1)
    j = abc[9].unsqueeze(1)
    modules[1]._ImplicitModule1Base__C = pol_order_3(implicit1_points, a, b, c, d, e, f, g, h, i, j).transpose(0, 1).unsqueeze(2)

    
# Initial coefficient for the polynomial model of the deformation constants
abc_init = torch.zeros(10, 2)
abc_init[0] = torch.ones(2)

In [10]:
# Creation of the atlas. Uses L2 norm attachment for now.
atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.L2NormAttachment()], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c_3)
#atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.VarifoldAttachment(2, [0.5, 2.])], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c)

In [11]:
# Fit the atlas
fitter = dm.Models.ModelFittingScipy(atlas, 1.)
costs = fitter.fit(targets, 150, options={'shoot_method': 'rk4', 'shoot_it': 10})

Initial energy = 2370489.000
Iteration: 1 
Total energy = 2201724.000000 
Attach cost = 2201713.250000 
Deformation cost = 10.886740
Iteration: 2 
Total energy = 2188061.000000 
Attach cost = 2188048.250000 
Deformation cost = 12.849781
Iteration: 3 
Total energy = 2163197.750000 
Attach cost = 2163183.500000 
Deformation cost = 14.409653
Iteration: 4 
Total energy = 2135833.000000 
Attach cost = 2135815.250000 
Deformation cost = 17.927057
Iteration: 5 
Total energy = 2113085.750000 
Attach cost = 2113066.000000 
Deformation cost = 19.825230
Iteration: 6 
Total energy = 2094452.750000 
Attach cost = 2094428.250000 
Deformation cost = 24.351013
Iteration: 7 
Total energy = 2070153.250000 
Attach cost = 2070124.625000 
Deformation cost = 28.550413
Iteration: 8 
Total energy = 1981523.250000 
Attach cost = 1981481.500000 
Deformation cost = 41.873753
Iteration: 9 
Total energy = 1893185.000000 
Attach cost = 1893129.500000 
Deformation cost = 55.636414
Iteration: 10 
Total energy = 18577

Iteration: 45 
Total energy = 126140.125000 
Attach cost = 124445.171875 
Deformation cost = 1694.954834
Iteration: 46 
Total energy = 122716.820312 
Attach cost = 121040.929688 
Deformation cost = 1675.903076
Iteration: 47 
Total energy = 115733.406250 
Attach cost = 114067.406250 
Deformation cost = 1665.997803
Iteration: 48 
Total energy = 108992.093750 
Attach cost = 107312.554688 
Deformation cost = 1679.530273
Iteration: 49 
Total energy = 104784.789062 
Attach cost = 103067.007812 
Deformation cost = 1717.785400
Iteration: 50 
Total energy = 102538.187500 
Attach cost = 100816.718750 
Deformation cost = 1721.465820
Iteration: 51 
Total energy = 100187.078125 
Attach cost = 98458.531250 
Deformation cost = 1728.541748
Iteration: 52 
Total energy = 98643.703125 
Attach cost = 96910.460938 
Deformation cost = 1733.234863
Iteration: 53 
Total energy = 95328.562500 
Attach cost = 93589.179688 
Deformation cost = 1739.380737
Iteration: 54 
Total energy = 92589.914062 
Attach cost = 90

KeyboardInterrupt: 

In [ ]:
plt.plot(range(len(costs)), costs)
plt.show()

In [13]:
# Retrieve the optimised coeffcient and then compare the generated growth constraint to the true one.
abc_fit = atlas.parameters[-1].detach()
print(0.1*abc_fit)
print(torch.tensor([[0., 0.3], [0., 1.], [0., 0.], [0., 0.], [0., 1.], [0., -1.] ]))

C_fit = pol_order_3(implicit1_points, abc_fit[0].unsqueeze(1), abc_fit[1].unsqueeze(1), abc_fit[2].unsqueeze(1), abc_fit[3].unsqueeze(1), abc_fit[4].unsqueeze(1), abc_fit[5].unsqueeze(1), abc_fit[6].unsqueeze(1), abc_fit[7].unsqueeze(1), abc_fit[8].unsqueeze(1), abc_fit[9].unsqueeze(1)).t().unsqueeze(2)

# The closer to 1. it is, the better.
print(torch.dot((true_C/torch.norm(true_C)).flatten(), (C_fit/torch.norm(C_fit)).flatten()))

tensor([[ 0.0924,  0.3517],
        [ 0.1370,  1.0985],
        [-0.0244,  0.0261],
        [-0.1259, -0.0035],
        [-0.0160,  1.1595],
        [ 0.0178, -1.1599],
        [ 0.0297,  0.2800],
        [ 0.0158,  0.0260],
        [-0.0516, -0.0040],
        [ 0.0028, -0.0100]])
tensor([[ 0.0000,  0.3000],
        [ 0.0000,  1.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  1.0000],
        [ 0.0000, -1.0000]])
tensor(0.9998)


In [15]:
ax0 = plt.subplot(111, aspect='equal')
dm.Utilities.plot_C_arrow(ax0, implicit1_points, true_C, R=implicit1_R, scale=0.05, zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, true_C, R=R, scale=1., color='black')

In [19]:
pts = atlas.models[0].init_manifold.gd[1][0].detach()
R = atlas.models[0].init_manifold.gd[1][1].detach()
ax1 = plt.subplot(111, aspect='equal')

dm.Utilities.plot_C_arrow(ax1, pts, C_fit, R=R, scale=0.05, zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C_fit, R=R, scale=1., color='black')

In [45]:
source = atlas.models[0].init_manifold.gd[0]

In [14]:
i=7
for i in range(10):
    shoot_method = 'rk4'
    shoot_it = 10
    silent_pos = atlas.models[i].init_manifold[0].gd.detach().clone()
    silent_mom = atlas.models[i].init_manifold[0].cotan.detach().clone()
    silent = dm.DeformationModules.SilentLandmarks(2, silent_pos.shape[0], gd=silent_pos, cotan=silent_mom)

    pts = atlas.models[i].init_manifold.gd[1][0].detach().clone()
    R = atlas.models[i].init_manifold.gd[1][1].detach().clone()
    imp1_cotan_pts = atlas.models[i].init_manifold.cotan[1][0].detach().clone()
    imp1_cotan_R = atlas.models[i].init_manifold.cotan[1][1].detach().clone()
    implicit1 = dm.DeformationModules.ImplicitModule1(2, pts.shape[0], sigma_implicit1, C_fit, nu=0.1, gd=(pts.clone().requires_grad_(), R.clone().requires_grad_()), cotan=(imp1_cotan_pts.clone().requires_grad_(), imp1_cotan_R.clone().requires_grad_()))


    h = dm.HamiltonianDynamic.Hamiltonian([silent, implicit1])
    intermediate_states, _ = dm.HamiltonianDynamic.shoot(h, shoot_it, shoot_method, intermediates=True)

    plt.figure()
    plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
    target = targets[i].detach().clone()

    t = -1
    transported_source = intermediate_states[t].gd[0].detach()
    plt.plot(transported_source[:, 0].numpy(), transported_source[:, 1].numpy(), color='red')
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), '--')

    plt.axis('equal')

In [60]:
plt.close('all')

In [114]:

points_per_side = 40

silent_points = dm.Utilities.generate_unit_square(points_per_side)
silent_points = silent_points*torch.tensor([1., 4.])
aabb = dm.Utilities.AABB.build_from_points(silent_points).scale(1.2)
implicit1_points = aabb.fill_uniform_density(20.)
implicit1_points = implicit1_points - torch.mean(implicit1_points, dim=0)
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)

def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

abc = torch.tensor([[0., 0.3], [0., 1.], [0., 0.], [0., 0.], [0., 1.], [0., -1.] ])
a = abc[0].unsqueeze(1)
b = abc[1].unsqueeze(1)
c = abc[2].unsqueeze(1)
d = abc[3].unsqueeze(1)
e = abc[4].unsqueeze(1)
f = abc[5].unsqueeze(1)
C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

#C = torch.zeros(implicit1_points.shape[0], 2, 1)
#C[:, 1, 0] = 10.*torch.abs(implicit1_points[:, 0] - torch.mean(implicit1_points, dim=0)[0] + 0.5*aabb.width)/aabb.width

ax1 = plt.subplot(111, aspect='equal')
plt.plot(silent_points[:,0], silent_points[:,1], '.b')
plt.plot(implicit1_points[:,0], implicit1_points[:,1], '.r')

dm.Utilities.plot_C_arrow(ax1, implicit1_points, C, R=implicit1_R, scale=1., zorder=3, mutation_scale=10)


In [115]:

def generate_bending():
    def _generate():
        silent_cotan = torch.randn_like(silent_points)
        implicit1_cotan = torch.zeros_like(implicit1_points)
        #val = 1000.*torch.rand(1) - 500.
        # val = 100.*torch.randn(1)

        # implicit1_cotan[0::7, 0] = val
        # implicit1_cotan[6::7, 0] = val
        #implicit1_cotan[6, 0] = val
        #implicit1_cotan[-1, 0] = 
        implicit1_cotan = 15.*torch.randn_like(implicit1_points)
        implicit1_cotan_R = torch.zeros_like(implicit1_R)

        # plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
        # plt.quiver(implicit1_points[:, 0].numpy(), implicit1_points[:, 1].numpy(), implicit1_cotan[:, 0].numpy(), implicit1_cotan[:, 1].numpy())
        # plt.axis('equal')
        # plt.show()

        # ax = plt.subplot()
        # plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
        # dm.Utilities.plot_C_arrow(ax, implicit1_points, C, scale=0.1, mutation_scale=10.)
        # plt.axis('equal')
        # plt.show()

        silent = dm.DeformationModules.SilentLandmarks(2, silent_points.shape[0], gd=silent_points.clone().requires_grad_(), cotan=silent_cotan.clone().requires_grad_())

        implicit1 = dm.DeformationModules.ImplicitModule1(2, implicit1_points.shape[0], 0.5, C, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()), cotan=(implicit1_cotan.clone().requires_grad_(), implicit1_cotan_R.clone().requires_grad_()))

        with torch.autograd.no_grad():
            dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 10, 'rk4')

        return silent.manifold.gd.detach(), implicit1.manifold.gd[0].detach()

    while True:
        bending = _generate()
        #print(1.5*torch.norm(silent_points), " ", torch.norm(bending[0]), " ", 4.*torch.norm(silent_points))
        if (not torch.any(torch.isnan(bending[0]))) and (torch.norm(bending[0]) >= 0.*torch.norm(silent_points)) and (torch.norm(bending[0]) <= 100.*torch.norm(silent_points)):
            print("New data element added. Frobenius norm: {norm}".format(norm=torch.norm(bending[0])))
            return bending


In [118]:
tar = generate_bending()

New data element added. Frobenius norm: 23.770769119262695


In [119]:

ax1 = plt.subplot(111, aspect='equal')
plt.plot(silent_points[:,0], silent_points[:,1], '.b')
plt.plot(implicit1_points[:,0], implicit1_points[:,1], '.r')

plt.plot(tar[0][:,0], tar[0][:,1], '-g')


tensor(21.4277)